# Goals
The goal of this notebook is to convert the historical files for each year and quarter into a small compressible format
Later on the information will be matched with the historical files data.
Furthermore this will include the code both for pandas and polars to serve as an initial benchmark for those to engines. Data will be stored in parquet.

you need to rewrite this to polars

In [115]:
import pyarrow
import pandas as pd
import polars as pl
import numpy as np
import sweetviz as sv
from collections import Counter
import numpy as np

In [93]:
pd.set_option('display.max_columns', None)

In [94]:
debug=True

In [95]:
# Assuming the columns in the dataset are in the order as follows, based on standard Freddie Mac file format
column_names = [
    "LOAN_SEQUENCE_NUMBER", "MONTHLY_REPORTING_PERIOD", "CURRENT_ACTUAL_UPB", "CURRENT_LOAN_DELINQUENCY_STATUS",
    "LOAN_AGE", "REMAINING_MONTHS", "REPURCHASE_FLAG", "MODIFICATION_FLAG", "ZERO_BALANCE_CODE",
    "ZERO_BALANCE_EFFECTIVE_DATE", "CURRENT_INTEREST_RATE", "CURRENT_DEFERRED_UPB", "DUE_DATE_OF_LAST_PAID_INSTALLMENT",
    "MI_RECOVERIES", "NET_SALES_PROCEEDS", "NON_MI_RECOVERIES", "EXPENSES", "LEGAL_COSTS", "MAINTENANCE_AND_PRESERVATION_COSTS",
    "TAXES_AND_INSURANCE", "MISCELLANEOUS_EXPENSES", "ACTUAL_LOSS_CALCULATION", "MODIFICATION_COST", "STEP_MODIFICATION_FLAG",
    "DEFERRED_PAYMENT_MODIFICATION", "ESTIMATED_LOAN_TO_VALUE", "ZERO_BALANCE_REMOVAL_UPB", "DELINQUENT_ACCRUED_INTEREST",
    "DELINQUENCY_DUE_TO_DISASTER", "BORROWER_ASSISTANCE_STATUS_CODE", "CURRENT_MONTH_LIQUIDATION_FLAG", "CURRENT_MONTH_REPURCHASE_FLAG"
]

In [96]:
column_datatypes = {
    "LOAN_SEQUENCE_NUMBER": pl.Utf8,
    "MONTHLY_REPORTING_PERIOD": pl.Utf8,
    "CURRENT_ACTUAL_UPB": pl.Float64,
    "CURRENT_LOAN_DELINQUENCY_STATUS": pl.Int64,
    "LOAN_AGE": pl.Int64,
    "REMAINING_MONTHS": pl.Int64,
    "REPURCHASE_FLAG": pl.Utf8,
    "MODIFICATION_FLAG": pl.Utf8,
    "ZERO_BALANCE_CODE": pl.Int64,
    "ZERO_BALANCE_EFFECTIVE_DATE": pl.Utf8,
    "CURRENT_INTEREST_RATE": pl.Float64,
    "CURRENT_DEFERRED_UPB": pl.Float64,
    "DUE_DATE_OF_LAST_PAID_INSTALLMENT": pl.Utf8,
    "MI_RECOVERIES": pl.Float64,
    "NET_SALES_PROCEEDS": pl.Float64,
    "NON_MI_RECOVERIES": pl.Float64,
    "EXPENSES": pl.Float64,
    "LEGAL_COSTS": pl.Float64,
    "MAINTENANCE_AND_PRESERVATION_COSTS": pl.Float64,
    "TAXES_AND_INSURANCE": pl.Float64,
    "MISCELLANEOUS_EXPENSES": pl.Float64,
    "ACTUAL_LOSS_CALCULATION": pl.Float64,
    "MODIFICATION_COST": pl.Float64,
    "STEP_MODIFICATION_FLAG": pl.Utf8,
    "DEFERRED_PAYMENT_MODIFICATION": pl.Utf8,
    "ESTIMATED_LOAN_TO_VALUE": pl.Float64,
    "ZERO_BALANCE_REMOVAL_UPB": pl.Float64,
    "DELINQUENT_ACCRUED_INTEREST": pl.Float64,
    "DELINQUENCY_DUE_TO_DISASTER": pl.Utf8,
    "BORROWER_ASSISTANCE_STATUS_CODE": pl.Utf8,
    "CURRENT_MONTH_LIQUIDATION_FLAG": pl.Utf8,
    "CURRENT_MONTH_REPURCHASE_FLAG": pl.Utf8
}


AttributeError: module 'pandas' has no attribute 'Utf8'

In [ ]:
def prepayment_for_Loan_Sequence_Number(df):
        """This function takes in a dataframe and returns a dataframe with a unique LOAN_Sequence_Number and if the customer prepaid at any point

        Args:
                df (_type_): _description_
        """
        # Assuming df_fixed_30yr and df are your DataFrames and are already defined

        # Initialize an empty list to store the results
        results = []

        df_fixed_30yr = df[(df['CURRENT_INTEREST_RATE'].notna()) & (df['REMAINING_MONTHS'] == 360)]

        for i in range(len(df_fixed_30yr)):
                # Extract the specific loan sequence number
                specific_loan_s_number = df_fixed_30yr["LOAN_SEQUENCE_NUMBER"].iloc[i]
                
                # Filter df for the current loan sequence number and non-null ZERO_BALANCE_CODES, then get unique values
                zero_balance_code_all_time = df[(df["LOAN_SEQUENCE_NUMBER"] == specific_loan_s_number) & df["ZERO_BALANCE_CODE"].notna()]["ZERO_BALANCE_CODE"].unique()
                
                # Determine if the loan was prepaid
                prepaid = 1 if 1 in zero_balance_code_all_time else 0
                
                # Append the result to the list
                results.append((specific_loan_s_number, prepaid))

        # Convert the list of results into a DataFrame
        prepaid_df = pd.DataFrame(results, columns=['LOAN_SEQUENCE_NUMBER', 'PREPAID'])

        return prepaid_df

In [ ]:
import polars as pl

def prepayment_for_loan_sequence_number_pl(df):
    """
    This function calculates the prepayment status for each loan sequence number using Polars.

    Args:
        df (pl.DataFrame): The input dataframe.

    Returns:
        pl.DataFrame: A dataframe with LOAN_SEQUENCE_NUMBER and prepayment status.
    """
    # Assuming 'ZERO_BALANCE_CODE' is the column indicating prepayment (code 1 for prepaid)
    prepayment_info = df.filter(pl.col("ZERO_BALANCE_CODE").is_not_null()).select([
        "LOAN_SEQUENCE_NUMBER",
        pl.col("ZERO_BALANCE_CODE").is_in([1]).alias("PREPAID")
    ]).groupby("LOAN_SEQUENCE_NUMBER").agg([
        pl.col("PREPAID").max().alias("PREPAID")
    ])
    
    return prepayment_info

def convert_parquet_pl(datapath='/Users/juliusgruber/Downloads/historical_data_2006/historical_data_time_2006Q1.txt', outputpath="data/hist_data_time_2006_Q1.parquet"):
    """
    Converts a CSV file to Parquet format with specific preprocessing using Polars.

    Args:
        datapath (str): Path to the input CSV file.
        outputpath (str): Path to the output Parquet file.
        debug (bool): If True, only loads the first 10,000 rows.
    """
    # Load the data
    if debug:
        df = pl.read_csv(datapath, separator='|', has_header=False, dtypes=column_datatypes, n_rows=10000)
    else:
        df = pl.read_csv(datapath, separator='|', has_header=False, dtypes=column_datatypes)

    # Filter the dataframe for fixed rate loans with original term equal to 360 months (30 years)
    df_fixed_30yr = df.filter((pl.col('CURRENT_INTEREST_RATE').is_not_null()) & (pl.col('REMAINING_MONTHS') == 360))
    
    relevant_features = ["LOAN_SEQUENCE_NUMBER", "MONTHLY_REPORTING_PERIOD", "CURRENT_ACTUAL_UPB", "ZERO_BALANCE_CODE", "CURRENT_INTEREST_RATE", "CURRENT_MONTH_REPURCHASE_FLAG"]
    
    final_df = df_fixed_30yr.select(relevant_features)

    # Calculate prepayment status
    prepayment_df = prepayment_for_loan_sequence_number_pl(df)
    final_df = final_df.join(prepayment_df, on='LOAN_SEQUENCE_NUMBER', how='left')

    # Write to Parquet
    final_df.write_parquet(outputpath)

# Column names, assuming they are provided as per the original question

# Example usage
convert_parquet()



ComputeError: could not parse `RA` as dtype `i64` at column 'CURRENT_LOAN_DELINQUENCY_STATUS' (column number 4)

The current offset in the file is 104567 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `dtypes` argument
- setting `ignore_errors` to `True`,
- adding `RA` to the `null_values` list.

Original error: ```remaining bytes non-empty```

In [108]:
datapath='/Users/juliusgruber/Downloads/historical_data_2006/historical_data_time_2006Q1.txt'

df = pd.read_csv(datapath, sep='|', header=None, low_memory=False)
df.columns = column_names

In [101]:
df

,LOAN_SEQUENCE_NUMBER,MONTHLY_REPORTING_PERIOD,CURRENT_ACTUAL_UPB,CURRENT_LOAN_DELINQUENCY_STATUS,LOAN_AGE,REMAINING_MONTHS,REPURCHASE_FLAG,MODIFICATION_FLAG,ZERO_BALANCE_CODE,ZERO_BALANCE_EFFECTIVE_DATE,CURRENT_INTEREST_RATE,CURRENT_DEFERRED_UPB,DUE_DATE_OF_LAST_PAID_INSTALLMENT,MI_RECOVERIES,NET_SALES_PROCEEDS,NON_MI_RECOVERIES,EXPENSES,LEGAL_COSTS,MAINTENANCE_AND_PRESERVATION_COSTS,TAXES_AND_INSURANCE,MISCELLANEOUS_EXPENSES,ACTUAL_LOSS_CALCULATION,MODIFICATION_COST,STEP_MODIFICATION_FLAG,DEFERRED_PAYMENT_MODIFICATION,ESTIMATED_LOAN_TO_VALUE,ZERO_BALANCE_REMOVAL_UPB,DELINQUENT_ACCRUED_INTEREST,DELINQUENCY_DUE_TO_DISASTER,BORROWER_ASSISTANCE_STATUS_CODE,CURRENT_MONTH_LIQUIDATION_FLAG,CURRENT_MONTH_REPURCHASE_FLAG
0,F06Q10000001,200603,130000.00,0,0,360,NaN,NaN,NaN,NaN,6.50,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130000.00
1,F06Q10000001,200604,130000.00,0,1,359,NaN,NaN,NaN,NaN,6.50,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130000.00
2,F06Q10000001,200605,130000.00,0,2,358,NaN,NaN,NaN,NaN,6.50,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130000.00
3,F06Q10000001,200606,130000.00,0,3,357,NaN,NaN,NaN,NaN,6.50,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130000.00
4,F06Q10000001,200607,129000.00,0,4,356,NaN,NaN,NaN,NaN,6.50,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,F06Q10000163,200709,226923.10,0,19,161,NaN,NaN,NaN,NaN,5.75,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,226923.10
9996,F06Q10000163,200710,225975.93,0,20,160,NaN,NaN,NaN,NaN,5.75,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225975.93
9997,F06Q10000163,200711,225024.22,0,21,159,NaN,NaN,NaN,NaN,5.75,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225024.22
9998,F06Q10000163,200712,224067.95,0,22,158,NaN,NaN,NaN,NaN,5.75,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224067.95


In [109]:
# Assuming 'df' is your DataFrame
compressed_df = df[df['ZERO_BALANCE_CODE'] == 1]


In [111]:
prepayed_list = compressed_df["LOAN_SEQUENCE_NUMBER"].values

In [ ]:

df_fixed_30yr = df[(df['CURRENT_INTEREST_RATE'].notna()) & (df['REMAINING_MONTHS'] == 360)]
    
relevant_features = ["LOAN_SEQUENCE_NUMBER", "MONTHLY_REPORTING_PERIOD","CURRENT_ACTUAL_UPB","ZERO_BALANCE_CODE","CURRENT_INTEREST_RATE","CURRENT_MONTH_REPURCHASE_FLAG" ]
get_now()
final_df = df_fixed_30yr[relevant_features].reset_index(drop=True)

#final_df['FINAL_STATUS'] = final_df['ZERO_BALANCE_CODE'].apply(lambda x: 1 if x == 1.0 else 0)
#string_counts = Counter(df["LOAN_SEQUENCE_NUMBER"].values)
#counts_df = pd.DataFrame(list(string_counts.items()), columns=['LOAN_SEQUENCE_NUMBER', 'COUNT'])
get_now()

#final_df = final_df.merge(prepayment_df, on='LOAN_SEQUENCE_NUMBER', how='left')
final_df["PREPAID"] = df["LOAN_SEQUENCE_NUMBER"].apply(lambda x:1 if x in prepayed_list else 0)
final_df.to_parquet(outputpath)  

In [113]:

df_fixed_30yr = df[(df['CURRENT_INTEREST_RATE'].notna()) & (df['REMAINING_MONTHS'] == 360)]
    
relevant_features = ["LOAN_SEQUENCE_NUMBER", "MONTHLY_REPORTING_PERIOD","CURRENT_ACTUAL_UPB","ZERO_BALANCE_CODE","CURRENT_INTEREST_RATE","CURRENT_MONTH_REPURCHASE_FLAG" ]
get_now()
final_df = df_fixed_30yr[relevant_features].reset_index(drop=True)

#final_df['FINAL_STATUS'] = final_df['ZERO_BALANCE_CODE'].apply(lambda x: 1 if x == 1.0 else 0)
#string_counts = Counter(df["LOAN_SEQUENCE_NUMBER"].values)
#counts_df = pd.DataFrame(list(string_counts.items()), columns=['LOAN_SEQUENCE_NUMBER', 'COUNT'])
get_now()

#final_df = final_df.merge(prepayment_df, on='LOAN_SEQUENCE_NUMBER', how='left')
#final_df["PREPAID"] = df["LOAN_SEQUENCE_NUMBER"].apply(lambda x:1 if x in prepayed_list else 0)

list_df = pd.DataFrame(prepayed_list, columns=['LOAN_SEQUENCE_NUMBER'])
list_df['indicator'] = 1

merged_df = pd.merge(final_df, list_df, on='LOAN_SEQUENCE_NUMBER', how='left').fillna({'indicator': 0})


#final_df.to_parquet(outputpath)  

08:05:04
08:05:04


In [ ]:
def convert_parquet(datapath='/Users/juliusgruber/Downloads/historical_data_2006/historical_data_time_2006Q1.txt', 
    outputpath="data/hist_data_time_2006_Q1.parquet"):
    get_now()
    # Load the data
    if debug==True:
        df = pd.read_csv(datapath, sep='|', header=None, nrows=10000, low_memory=False)
    else:
        df = pd.read_csv(datapath, sep='|', header=None, low_memory=False)
    
    df.columns = column_names
    #fuckup, you need to check for loans that are countinously payed, you can't forward fill them in this way
    #df['ZERO_BALANCE_CODE'] = df['ZERO_BALANCE_CODE'].bfill()
    
    # Filter the dataframe for fixed rate (assuming 'CURRENT_INTEREST_RATE' is non-null for fixed rate)
    # and loans with original term equal to 360 months (30 years)
    df_fixed_30yr = df[(df['CURRENT_INTEREST_RATE'].notna()) & (df['REMAINING_MONTHS'] == 360)]
    
    relevant_features = ["LOAN_SEQUENCE_NUMBER", "MONTHLY_REPORTING_PERIOD","CURRENT_ACTUAL_UPB","ZERO_BALANCE_CODE","CURRENT_INTEREST_RATE","CURRENT_MONTH_REPURCHASE_FLAG" ]
    get_now()
    final_df = df_fixed_30yr[relevant_features].reset_index(drop=True)
    
    #final_df['FINAL_STATUS'] = final_df['ZERO_BALANCE_CODE'].apply(lambda x: 1 if x == 1.0 else 0)
    #string_counts = Counter(df["LOAN_SEQUENCE_NUMBER"].values)
    #counts_df = pd.DataFrame(list(string_counts.items()), columns=['LOAN_SEQUENCE_NUMBER', 'COUNT'])
    get_now()
    prepayment_df = prepayment_for_Loan_Sequence_Number(df)
    final_df = final_df.merge(prepayment_df, on='LOAN_SEQUENCE_NUMBER', how='left')
    final_df.to_parquet(outputpath)  

In [114]:
merged_df

,LOAN_SEQUENCE_NUMBER,MONTHLY_REPORTING_PERIOD,CURRENT_ACTUAL_UPB,ZERO_BALANCE_CODE,CURRENT_INTEREST_RATE,CURRENT_MONTH_REPURCHASE_FLAG,indicator
0,F06Q10000001,200603,130000.00,NaN,6.500,130000.00,1.0
1,F06Q10000002,200603,214000.00,NaN,6.250,214000.00,1.0
2,F06Q10000003,200603,81000.00,NaN,6.375,81000.00,1.0
3,F06Q10000005,200604,296000.00,NaN,6.250,296000.00,1.0
4,F06Q10000006,200602,255000.00,NaN,6.250,255000.00,1.0
...,...,...,...,...,...,...,...
207178,F06Q10373684,201602,165694.96,NaN,5.875,165694.96,1.0
207179,F06Q10373697,200905,150000.00,NaN,5.000,150000.00,1.0
207180,F06Q10373781,200908,146000.00,NaN,5.200,146000.00,1.0
207181,F06Q10373947,202210,101710.78,NaN,3.375,96510.78,0.0


In [129]:
def convert_parquet_final(datapath='/Users/juliusgruber/Downloads/historical_data_2006/historical_data_time_2006Q1.txt', 
    outputpath="data/hist_data_time_2006_Q1.parquet"):
    get_now()
    # Load the data
    if debug==True:
        df = pd.read_csv(datapath, sep='|', header=None, nrows=10000, low_memory=False)
    else:
        df = pd.read_csv(datapath, sep='|', header=None, low_memory=False)
    get_now()
    df.columns = column_names
    df_fixed_30yr = df[(df['CURRENT_INTEREST_RATE'].notna()) & (df['REMAINING_MONTHS'] == 360)]
        
    relevant_features = ["LOAN_SEQUENCE_NUMBER", "MONTHLY_REPORTING_PERIOD","CURRENT_ACTUAL_UPB","ZERO_BALANCE_CODE","CURRENT_INTEREST_RATE","CURRENT_MONTH_REPURCHASE_FLAG" ]
    final_df = df_fixed_30yr[relevant_features].reset_index(drop=True)

    list_df = pd.DataFrame(prepayed_list, columns=['LOAN_SEQUENCE_NUMBER'])
    list_df['indicator'] = 1

    merged_df = pd.merge(final_df, list_df, on='LOAN_SEQUENCE_NUMBER', how='left').fillna({'indicator': 0})

    #find out how long until prepayment
    string_counts = Counter(df["LOAN_SEQUENCE_NUMBER"].values)
    counts_df = pd.DataFrame(list(string_counts.items()), columns=['LOAN_SEQUENCE_NUMBER', 'COUNT'])

    merged_df = pd.merge(final_df, counts_df, on='LOAN_SEQUENCE_NUMBER', how='left')

    merged_df = merged_df.dropna(subset=['CURRENT_INTEREST_RATE'])

    merged_df.to_parquet(outputpath)  

In [119]:
merged_df

,LOAN_SEQUENCE_NUMBER,MONTHLY_REPORTING_PERIOD,CURRENT_ACTUAL_UPB,ZERO_BALANCE_CODE,CURRENT_INTEREST_RATE,CURRENT_MONTH_REPURCHASE_FLAG,indicator
0,F06Q10000001,200603,130000.00,NaN,6.500,130000.00,1.0
1,F06Q10000002,200603,214000.00,NaN,6.250,214000.00,1.0
2,F06Q10000003,200603,81000.00,NaN,6.375,81000.00,1.0
3,F06Q10000005,200604,296000.00,NaN,6.250,296000.00,1.0
4,F06Q10000006,200602,255000.00,NaN,6.250,255000.00,1.0
...,...,...,...,...,...,...,...
207178,F06Q10373684,201602,165694.96,NaN,5.875,165694.96,1.0
207179,F06Q10373697,200905,150000.00,NaN,5.000,150000.00,1.0
207180,F06Q10373781,200908,146000.00,NaN,5.200,146000.00,1.0
207181,F06Q10373947,202210,101710.78,NaN,3.375,96510.78,0.0


In [123]:
debug=False

In [130]:
%%time

# Iterate over each year and quarter
for year in range(2006, 2023):  # 2024 because the range end is exclusive
    for quarter in range(1, 5):  # From Q1 to Q4
        print(year,quarter)
        
        # Construct the file path
        file_path = f'/Users/juliusgruber/Downloads/historical_data_{year}/historical_data_time_{year}Q{quarter}.txt'
        output_path = f'data/hist_data_time_{year}Q{quarter}.parquet'
        try:
            convert_parquet_final(file_path,output_path)
        except Exception as e:
            # If there's an error reading a file, print the error message and file path
            print(f"Could not read {file_path}: {e}")

2006 1
08:36:15
08:37:42
2006 2
08:37:44
08:39:06
2006 3
08:39:08
08:40:20
2006 4
08:40:21
08:41:48
2007 1
08:41:49
08:43:11
2007 2
08:43:12
08:44:49
2007 3
08:44:51
08:45:34
2007 4
08:45:35
08:46:44
2008 1
08:46:46
08:48:25
2008 2
08:48:27
08:49:37
2008 3
08:49:38
08:49:48
2008 4
08:49:48
08:49:58
2009 1
08:49:59
08:52:41
2009 2
08:52:45
08:57:09
2009 3
08:57:14
08:59:16
2009 4
08:59:19
09:01:19
2010 1
09:01:22
09:02:54
2010 2
09:02:56
09:04:26
2010 3
09:04:28
09:07:08
2010 4
09:07:12
09:11:14
2011 1
09:11:17
09:12:53
2011 2
09:12:55
09:13:23
2011 3
09:13:24
09:15:07
2011 4
09:15:09
09:17:30
2012 1
09:17:33
09:20:10
2012 2
09:20:13
09:23:10
2012 3
09:23:14
09:37:37
2012 4
09:37:42
09:57:31
2013 1
09:57:36
10:03:18
2013 2
10:03:23
10:07:30
2013 3
10:07:34
10:09:19
2013 4
10:09:21
10:10:05
2014 1
10:10:07
10:10:24
2014 2
10:10:25
10:10:48
2014 3
10:10:50
10:12:06
2014 4
10:12:07
10:12:55
2015 1
10:12:56
10:14:26
2015 2
10:14:28
10:16:09
2015 3
10:16:11
10:17:32
2015 4
10:17:34
10:18:17


In [82]:
from datetime import datetime
def get_now():
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(current_time)

In [61]:
pd.read_parquet('data/hist_data_time_2006Q1.parquet')

,LOAN_SEQUENCE_NUMBER,MONTHLY_REPORTING_PERIOD,CURRENT_ACTUAL_UPB,ZERO_BALANCE_CODE,CURRENT_INTEREST_RATE,CURRENT_MONTH_REPURCHASE_FLAG,PREPAID
0,F06Q10000001,200603,130000.0,NaN,6.500,130000.0,1
1,F06Q10000002,200603,214000.0,NaN,6.250,214000.0,1
2,F06Q10000003,200603,81000.0,NaN,6.375,81000.0,1
3,F06Q10000005,200604,296000.0,NaN,6.250,296000.0,1
4,F06Q10000006,200602,255000.0,NaN,6.250,255000.0,1
...,...,...,...,...,...,...,...
105,F06Q10000155,200602,158000.0,NaN,6.500,158000.0,1
106,F06Q10000156,200602,176000.0,NaN,6.250,176000.0,1
107,F06Q10000157,200603,93000.0,NaN,6.500,93000.0,0
108,F06Q10000158,200602,100000.0,NaN,5.375,100000.0,1


In [83]:
get_now()

21:48:55


# Statistical Analysis of Data

Mainly check the percentage of people that prepay in total over time. 
Then look at which time they prepay and how the distribution looks like over time

With this we want to filter, when we should start our testing period for our model and how much data we can include.